In [ ]:
"""
Часть 1

Нужно написать скрипт, который скачивает все данные прошедших президентских выборов для всех избирательных участков.

Входная точка по ссылке. 
http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1&tvd=100100084849066&vrn=100100084849062&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100084849066&type=227
Затем нужно перейти на сайты региональных избирательных комиссий. 
Результаты нужно сохранить в cvs-файл, sqlite базе данных или parquet-файле. 
В итоге должна получиться таблица с полями: 
- название региона 
- название ТИК 
- номер УИК 
- 20 стандартных полей из итогового протокола


Часть 2

Нужно, используя Spark: 
- найти явку (%) по всем регионам, результат отсортировать по убыванию 
- выбрать любимого кандидата и найти тот избиратльный участок, 
на котором он получил наибольший результат (учитывать участки на которых проголосовало больше 300 человек) 
- найти регион, где разница между ТИК с наибольшей явкой и наименьшей максимальна 
- посчитать дисперсию по явке для каждого региона (учитывать УИК) 
- для каждого кандидата посчитать таблицу: результат (%, округленный до целого) 
- количество УИК, на которых кандидат получил данный результат

Результаты принимаются в виде Jupyter Notebook, Spark Notebook или исходных файлов на Scala.

"""

In [1]:
import sqlite3
import lxml.html
import lxml.etree
import requests
import re

In [2]:
def create_table():
    conn = sqlite3.connect("elections.db")
    cursor = conn.cursor()
    cursor.execute("create table if not exists elections (region text, tik text, uik text)")
    for i in range(1, 21):
        cursor.execute("alter table elections add v_%i integer" % i)
    for i in range(13, 21):
        cursor.execute("alter table elections add percent_%i real" %i)
    conn.commit()
    conn.close()

In [3]:
def get_titles(url):
    response = requests.get(url)
    tree = lxml.html.fromstring(response.text)
    data = tree.xpath('//td[contains(@width, "90%")]//tr[contains(@valign, "top")]//a')
    return [{"Name": x.text, "URL": x.attrib["href"]} for x in data]

In [4]:
def add_data(url, region, tik):
    tree = lxml.html.fromstring(requests.get(url).text)
    url = tree.xpath('//a[contains(text(), "сайт избирательной комиссии субъекта Российской Федерации")]')[0].attrib["href"]
    tree = lxml.html.fromstring(requests.get(url).text)
    data = tree.xpath('//table[contains(@align, "left")]//tr[count(td)=3]')
    uiks = [x.text for x in tree.xpath('//td[contains(@width, "90%")]/div/table//tr[contains(@valign, "top")]/td/nobr')]
    data = tree.xpath(f"//td[contains(@width, '90%')]/div/table//tr[count(td)={len(uiks)} and not(contains(@valign, 'top')) and td[count(nobr)>0]]")
    rows = {}
    conn = sqlite3.connect("elections.db")
    cursor = conn.cursor()
    for uik_index, uik in enumerate(uiks):
        request = f'insert into elections values ("{region}", "{tik}", "{uik}"'
        current = {"val": [], "percent": []}
        for row in range(1, 21):
            cell = data[row-1].getchildren()[uik_index]
            percent = re.findall(r"\w+[.]\w+[%]", cell.text_content())
            if percent:
                current['percent'].append((percent[0][:-1]))
            current['val'].append((cell.xpath("./nobr/b")[0].text))
        for val in current['val']:
            request += f", {val}"
        for percent in current['percent']:
            request += f", {percent}"
        request += ")"
        cursor.execute(request)
        conn.commit()

In [7]:
def fill_db():
    create_table()
    url = 'http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1&tvd=100100084849066&vrn=100100084849062&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100084849066&type=227'
    regions = get_titles(url)
    for region in regions:
        print(region["Name"])
        tiks = get_titles(region["URL"])
        for tik in tiks:
            add_data(tik["URL"], region["Name"], tik["Name"])

In [6]:
fill_db()

Республика Адыгея (Адыгея)
Республика Алтай
Республика Башкортостан
Республика Бурятия
Республика Дагестан
Республика Ингушетия
Кабардино-Балкарская Республика
Республика Калмыкия
Карачаево-Черкесская Республика
Республика Карелия
Республика Коми
Республика Крым
Республика Марий Эл
Республика Мордовия
Республика Саха (Якутия)
Республика Северная Осетия - Алания
Республика Татарстан (Татарстан)
Республика Тыва
Удмуртская Республика
Республика Хакасия
Чеченская Республика
Чувашская Республика - Чувашия
Алтайский край
Забайкальский край
Камчатский край
Краснодарский край
Красноярский край
Пермский край
Приморский край
Ставропольский край
Хабаровский край
Амурская область
Архангельская область
Астраханская область
Белгородская область
Брянская область
Владимирская область
Волгоградская область
Вологодская область
Воронежская область
Ивановская область
Иркутская область
Калининградская область
Калужская область
Кемеровская область
Кировская область
Костромская область
Курганская область
Кур